### 문제
#### https://school.programmers.co.kr/learn/courses/30/lessons/42860

```txt
문제 설명
조이스틱으로 알파벳 이름을 완성하세요. 맨 처음엔 A로만 이루어져 있습니다.
ex) 완성해야 하는 이름이 세 글자면 AAA, 네 글자면 AAAA

조이스틱을 각 방향으로 움직이면 아래와 같습니다.

▲ - 다음 알파벳
▼ - 이전 알파벳 (A에서 아래쪽으로 이동하면 Z로)
◀ - 커서를 왼쪽으로 이동 (첫 번째 위치에서 왼쪽으로 이동하면 마지막 문자에 커서)
▶ - 커서를 오른쪽으로 이동 (마지막 위치에서 오른쪽으로 이동하면 첫 번째 문자에 커서)

예를 들어 아래의 방법으로 "JAZ"를 만들 수 있습니다.

- 첫 번째 위치에서 조이스틱을 위로 9번 조작하여 J를 완성합니다.
- 조이스틱을 왼쪽으로 1번 조작하여 커서를 마지막 문자 위치로 이동시킵니다.
- 마지막 위치에서 조이스틱을 아래로 1번 조작하여 Z를 완성합니다.
따라서 11번 이동시켜 "JAZ"를 만들 수 있고, 이때가 최소 이동입니다.

만들고자 하는 이름 name이 매개변수로 주어질 때, 이름에 대해 조이스틱 조작 횟수의 최솟값을 return 하도록 solution 함수를 만드세요.

제한 사항
- name은 알파벳 대문자로만 이루어져 있습니다.
- name의 길이는 1 이상 20 이하입니다.

입출력 예
name	return
"JEROEN"	56
"JAN"	23
```


---

### 풀이
```txt
A - Z가 이어진 구조
1. 문자 변경
ord('A'), ord('Z')에서 어느쪽에 더 가까운지 판별하면 된다.
이동수는 다음과 같이 구한다.
- 순방향 : ord(c) - ord('A')
- 역방향 : ord('Z') - ord(c) + 1
이중 작은 방향으로 이동하는 걸 더한다. 

2. 위치 변경
최적 커서 경로는 턴을 0~1번만 하면 충분한 구조이고,
그 턴이 일어나는 위치를 i로 두면 모든 최적 후보가 그 안에 포함되므로,
i위치에서 언제 턴할지 전부 비교하는 방식이 최소 이동을 보장한다.
* 왜 턴을 안 하거나 한 번만 턴하는 게 최적이냐면, 왼쪽 오른쪽 왔다갔다를 생각해보면 손해라는 걸 알 수 있다. 그래서 0 ~ 1번 턴을 해야 한다.

- 2 * i + (nLen - nNext) : 오른쪽으로 i까지 갔다가 돌아와서(턴) 뒤쪽(끝쪽) 처리
- i + 2 * (nLen - nNext) : 뒤쪽(끝쪽)부터 먼저 처리하고 돌아와서(턴) i까지 처리
여기서 nNext는 i+1부터 연속된 'A'를 쭉 건너뛴 다음의 첫 바꿔야 할 위치이다.

그림으로 표현하자면
오른쪽으로 i까지 갔다가 돌아와서(턴) 뒤쪽(끝쪽) 처리
                  시작------------------>(턴)
<--------------------------------------
|   nLen - nNext   |       i * 2        |

뒤쪽(끝쪽)부터 먼저 처리하고 돌아와서(턴) i까지 처리
(턴)<-----------------시작
    --------------------------------------->
    | 2 * (nLen - nNext) |         i         |
```


#### <br/>
#### 아래는 이전에 생각했던 방법인데, AABB와 같은 케이스에서 깨진다.
```txt
2. 위치 변경
A 이면 변경안 해도 됨
다음 바꿀 타겟이 <-, -> 둘 중 어디에 가까운지를 알아야 한다.
JAZ인 경우 J를 먼저 바꿨고, A는 변경할 필요 없고, Z는 <- 쪽이 1번만 이동하면 된다.

JEROEN의 경우 A 문자가 없으므로 다음 문자를 계속 보고 -> 으로만 이동하면 된다.

ZZAAAAAAAZ 가 있다고 해보자. 이건 Z 바꾸고 / -> / Z 바꾸고 / <- <- / Z 바꾸는 방법으로 이동해야 한다. 

ZZAAAZAAAZ 는 Z / -> / Z / -> -> -> -> / Z / -> -> -> -> / Z 이다. 

A가 아닌 다음 문자의 위치만 고려하면 된다. 

len(s) - 1은 -1과 같다. 인덱스를 마이너스로 만들려면 i - (len(s) - 1) - 1 = i - len(s)로 하면 된다.
플러스 방향이 가까운지, 마이너스 방향이 가까운지는 이렇게 계산한다.
이전 문자 인덱스를 nPrev_idx, 현재 문자 인덱스를 i라 하면
- 플러스 방향 이동수 : i - nPrev_idx
- 마이너스 방향 이동수 : abs(i - len(s)) + nPrev_idx
이중 작은 방향으로 이동하는 걸 더한다.
```

In [ ]:
def solution(name):
    nLen = len(name)
    nOrd_a = ord('A')
    nOrd_z = ord('Z')

    nVertical = 0
    nMove_min = nLen - 1

    for i in range(0, nLen):
        # 문자 변경은 그냥 누적
        sCh = name[i]
        nUp = ord(sCh) - nOrd_a
        nDown = nOrd_z - ord(sCh) + 1
        nVertical += nUp if nUp <= nDown else nDown
        
        # 자리 이동
        nNext = i + 1
        while nNext < nLen and name[nNext] == 'A':
            nNext += 1

        nCase1 = 2 * i + (nLen - nNext)
        nCase2 = i + 2 * (nLen - nNext)
        if nCase1 < nMove_min:
            nMove_min = nCase1
        if nCase2 < nMove_min:
            nMove_min = nCase2

    answer = nVertical + nMove_min
    return answer


name = "JAN"
result = solution(name)
print(result)

name = "JAZ"
result = solution(name)
print(result)

name = "JEROEN"
result = solution(name)
print(result)

name = "ZZAAAAAAAZ"
result = solution(name)
print(result)

name = "ZZAAAZAAAZ"
result = solution(name)
print(result)

name = "AABB"
result = solution(name)
print(result)


23
11
56
6
11
4


```
정확성  테스트
테스트 1 〉	통과 (0.01ms, 9.11MB)
테스트 2 〉	통과 (0.01ms, 9.16MB)
테스트 3 〉	통과 (0.01ms, 9.31MB)
테스트 4 〉	통과 (0.01ms, 9.29MB)
테스트 5 〉	통과 (0.01ms, 9.24MB)
테스트 6 〉	통과 (0.01ms, 9.31MB)
테스트 7 〉	통과 (0.01ms, 9.19MB)
테스트 8 〉	통과 (0.01ms, 9.07MB)
테스트 9 〉	통과 (0.01ms, 9.19MB)
테스트 10 〉	통과 (0.01ms, 9.3MB)
테스트 11 〉	통과 (0.01ms, 9.18MB)
테스트 12 〉	통과 (0.01ms, 9.16MB)
테스트 13 〉	통과 (0.01ms, 9.29MB)
테스트 14 〉	통과 (0.01ms, 9.28MB)
테스트 15 〉	통과 (0.03ms, 9.11MB)
테스트 16 〉	통과 (0.01ms, 9.11MB)
테스트 17 〉	통과 (0.01ms, 9.19MB)
테스트 18 〉	통과 (0.01ms, 9.3MB)
테스트 19 〉	통과 (0.01ms, 9.39MB)
테스트 20 〉	통과 (0.02ms, 9.12MB)
테스트 21 〉	통과 (0.01ms, 9.11MB)
테스트 22 〉	통과 (0.02ms, 9.24MB)
테스트 23 〉	통과 (0.02ms, 9.25MB)
테스트 24 〉	통과 (0.01ms, 9.07MB)
테스트 25 〉	통과 (0.03ms, 9.24MB)
테스트 26 〉	통과 (0.03ms, 9.27MB)
테스트 27 〉	통과 (0.01ms, 9.3MB)
```


In [ ]:
print(ord('Z') - ord('A'))
print((ord('Z') - ord('A')) // 2)
print((ord('B') - ord('A')))
print(ord('Z') - ord('Y') + 1)

s = "abc"
print(s[0 - len(s)])

# ABCDEFGHIJKLMNOPQRSTUVWXYZ

25
12
1
2
a
